In [5]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

In [6]:
pd_JH_data=pd.read_csv('C:\\Users\\mouni\\OneDrive\\Desktop\\DataScience\\data\\processed\\COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,Kosovo,0.0
2,2020-01-22,no,Kuwait,0.0
3,2020-01-22,no,Kyrgyzstan,0.0
4,2020-01-22,no,Laos,0.0


In [7]:
test_data=pd_JH_data[((pd_JH_data['country']=='Bhutan')|
                      (pd_JH_data['country']=='Denmark'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [8]:
test_data.head()

,date,state,country,confirmed
17020,2020-03-21,Greenland,Denmark,2.0
17026,2020-03-21,Faroe Islands,Denmark,92.0
17044,2020-03-21,no,Bhutan,2.0
17054,2020-03-21,no,Denmark,1326.0
17297,2020-03-22,Faroe Islands,Denmark,115.0


In [9]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Bhutan,2022-07-28,no,60368.0
Denmark,2022-07-28,no,3228291.0


In [10]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

In [11]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state         country                      
Faroe Islands Denmark 2022-07-28    34658.0
Greenland     Denmark 2022-07-28    11971.0
no            Bhutan  2022-07-28    60368.0
              Denmark 2022-07-28  3228291.0

In [12]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [13]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state          country        
Faroe Islands  Denmark  17026             NaN
                        17297             NaN
                        17595        8.333333
                        17867       33.809524
                        18166       17.714286
                                     ...     
no             Denmark  260731            inf
                        261016            inf
                        261301     992.474987
                        261586     776.803872
                        261868    1907.560548
Name: confirmed, Length: 3440, dtype: float64

In [14]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [15]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR', 'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,499,NaN
2,Alberta,Canada,762,NaN
3,Alberta,Canada,1048,NaN
4,Alberta,Canada,1354,NaN


In [16]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,Kosovo,0.0
2,2,2020-01-22,no,Kuwait,0.0
3,3,2020-01-22,no,Kyrgyzstan,0.0
4,4,2020-01-22,no,Laos,0.0


In [17]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,Kosovo,0.0,NaN
2,2,2020-01-22,no,Kuwait,0.0,NaN
3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN
4,4,2020-01-22,no,Laos,0.0,NaN


In [18]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [19]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [20]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.head()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0
1,1,2020-01-22,no,Kosovo,0.0,NaN,0.0
2,2,2020-01-22,no,Kuwait,0.0,NaN,0.0
3,3,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0
4,4,2020-01-22,no,Laos,0.0,NaN,0.0


In [21]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling.tail()

,state,country,index,confirmed_filtered_DR
261910,no,Zimbabwe,260581,17082.648889
261911,no,Zimbabwe,260823,15530.682828
261912,no,Zimbabwe,261151,13631.556738
261913,no,Zimbabwe,261393,13177.005998
261914,no,Zimbabwe,261914,12263.765550


In [22]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
261910,261910,2022-07-28,no,Andorra,45508.0,498.754579,45435.2,1.247220e+03
261911,261911,2022-07-28,no,Algeria,267194.0,2903.260870,267189.0,3.001124e+03
261912,261912,2022-07-28,no,Albania,309278.0,247.520289,309166.6,2.631772e+02
261913,261913,2022-07-28,no,Argentina,9507562.0,inf,9507562.0,-1.020867e+16
261914,261914,2022-07-28,no,Zimbabwe,256336.0,9858.141026,256333.6,1.226377e+04


In [23]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 

In [24]:
pd_result_larg[pd_result_larg['country']=='Denmark'].tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
261556,261556,2022-07-27,Faroe Islands,Denmark,34658.0,inf,34658.0,9.526718e+15
261586,261586,2022-07-27,no,Denmark,3226717.0,776.803872,3226154.9,1.349052e+03
261834,261834,2022-07-28,Greenland,Denmark,11971.0,inf,11971.0,2.794369e+31
261840,261840,2022-07-28,Faroe Islands,Denmark,34658.0,inf,34658.0,-2.022539e+31
261868,261868,2022-07-28,no,Denmark,3228291.0,1907.560548,3228959.4,1.150349e+03


In [26]:
pd_result_larg.to_csv(r'C:\Users\mouni\OneDrive\Desktop\DataScience\data\processed\COVID_final_set.csv',sep=';',index=False)